In [1]:
from llm_opt_toolkit.prompt_builder import build_prompt
from llm_opt_toolkit.optimization_journal import  Journal
from llm_opt_toolkit.llm_requester import call_llm
from IPython.display import Markdown, display
from config import GRIDS, InformerConfig, build_llm_list_config, get_train_config
from pprint import pprint

grid = GRIDS['grid12']
journal = Journal(filename='ETTh2-pred_len=24;th_b=8192;hypot.json')
history_table = journal.get_history_table()
prompt = build_prompt(grid, 1, history_table)
ListConfigs = build_llm_list_config(grid)

In [2]:
scheme = call_llm(prompt, ListConfigs, temperature=0.7, thinking_budget=8192)
cfg = scheme[0]
pprint(cfg.model_dump())

{'e_layers': 2,
 'factor': 8,
 'hypothesis': 'Мы тестируем, может ли более высокий фактор (8) улучшить '
               'обобщение на оптимальном коротком контексте (48ч) и '
               'высокопараллельной/низкоглубинной архитектуре (2 слоя, 16 '
               'голов). Это проверяет, может ли более широкий, но быстрый '
               'отбор зависимостей обеспечить лучшее обобщение, чем '
               'сфокусированное внимание (factor 3/5).',
 'label_len': 24,
 'n_heads': 16,
 'seq_len': 48}


In [3]:
train_cfg = get_train_config(cfg, pred_len=24)
pprint(train_cfg.model_dump())

{'activation': 'gelu',
 'attn': 'prob',
 'batch_size': 32,
 'c_out': 1,
 'checkpoints': './informer_checkpoints',
 'd_ff': 2048,
 'd_layers': 1,
 'd_model': 512,
 'data': 'custom',
 'dec_in': 7,
 'des': 'llm_search',
 'detail_freq': 'h',
 'device_ids': [],
 'devices': '',
 'distil': True,
 'dropout': 0.05,
 'e_layers': 2,
 'embed': 'timeF',
 'enc_in': 7,
 'factor': 8,
 'features': 'MS',
 'freq': 'h',
 'gpu': None,
 'hypothesis': 'Мы тестируем, может ли более высокий фактор (8) улучшить '
               'обобщение на оптимальном коротком контексте (48ч) и '
               'высокопараллельной/низкоглубинной архитектуре (2 слоя, 16 '
               'голов). Это проверяет, может ли более широкий, но быстрый '
               'отбор зависимостей обеспечить лучшее обобщение, чем '
               'сфокусированное внимание (factor 3/5).',
 'itr': 1,
 'label_len': 24,
 'learning_rate': 0.0001,
 'loss': 'mse',
 'lradj': 'type1',
 'mix': True,
 'model': 'informer',
 'n_heads': 16,
 'num_workers': 

In [4]:
params_str = ' '.join(f"{k}={v}" for k, v in cfg.__dict__.items() if k != 'hypothesis')
print(f"[1/10] {params_str}", end=' ')
# [1/10] seq_len=48 label_len=24 e_layers=3 n_heads=16 factor=5
# [1/10] seq_len=48 label_len=24 e_layers=3 n_heads=16 factor=10 d_model=768 d_ff=3072 d_layers=1 dropout=0.1

[1/10] seq_len=48 label_len=24 e_layers=2 n_heads=16 factor=8 

In [5]:
from llm_opt_toolkit.train import train_informer

In [ ]:
measures = train_informer('/Users/andrejustinov/Desktop/nas_project/data/ETTh1_prepared.csv', train_cfg, 24)